In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2, l1
from keras import backend as K

In [7]:
data_PL23 = pd.read_csv('elo/csv/England/2023/PL.csv')
data_kubokPL23 = pd.read_csv('elo/csv/England/2023/cup.csv')
data_kubokPL23 = data_kubokPL23[data_kubokPL23['home_team_id'].isin(data_PL23['home_team_id'].unique()) & data_kubokPL23['away_team_id'].isin(
                 data_PL23['home_team_id'].unique())]
data_carabaoPL23 = pd.read_csv('elo/csv/England/2023/carabao.csv')
data_carabaoPL23 = data_carabaoPL23[data_carabaoPL23['home_team_id'].isin(data_PL23['home_team_id'].unique()) & data_carabaoPL23['away_team_id'].isin(
                 data_PL23['home_team_id'].unique())]
data_PL23 = data_PL23.merge(data_kubokPL23, how='outer')
data_PL23 = data_PL23.merge(data_carabaoPL23, how='outer')
data_PL23.sort_values('timestamp', inplace=True)
data_PL23.reset_index(inplace=True, drop=True)
data_PL23

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,1035037,1691780400,44,Burnley,0,3,50,Manchester City
1,1035038,1691839800,42,Arsenal,2,1,65,Nottingham Forest
2,1035039,1691848800,35,Bournemouth,1,1,48,West Ham
3,1035040,1691848800,51,Brighton,4,1,1359,Luton
4,1035041,1691848800,45,Everton,0,1,36,Fulham
...,...,...,...,...,...,...,...,...
126,1035150,1699797600,51,Brighton,1,1,62,Sheffield Utd
127,1035155,1699797600,48,West Ham,3,2,65,Nottingham Forest
128,1035149,1699797600,66,Aston Villa,3,1,36,Fulham
129,1035153,1699797600,40,Liverpool,3,0,55,Brentford


In [10]:
data_RFPL23 = pd.read_csv('elo/csv/Russia/2023/RFPL.csv')
data_kubokRFPL23 = pd.read_csv('elo/csv/Russia/2023/KUBOK.csv')
data_kubokRFPL23 = data_kubokRFPL23[data_kubokRFPL23['home_team_id'].isin(data_RFPL23['home_team_id'].unique()) & data_kubokRFPL23['away_team_id'].isin(
                 data_RFPL23['home_team_id'].unique())]
data_RFPL23 = data_RFPL23.merge(data_kubokRFPL23, how='outer').sort_values('timestamp').reset_index(drop=True)
data_RFPL23.reset_index(inplace=True, drop=True)
data_RFPL23

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,1042016,1689955200,1088,Dinamo Moscow,1.0,3.0,621,Krasnodar
1,1042021,1690020000,1084,Ural,2.0,1.0,555,CSKA Moscow
2,1042020,1690028100,2011,Nizhny Novgorod,0.0,2.0,596,Zenit Saint Petersburg
3,1042017,1690036200,597,Lokomotiv Moscow,2.0,2.0,1083,Rubin
4,1042018,1690045200,1085,Akhmat Grozny,1.0,2.0,1079,Krylya Sovetov
...,...,...,...,...,...,...,...,...
163,1042126,1699720200,596,Zenit Saint Petersburg,1.0,1.0,621,Krasnodar
164,1042132,1699786800,2011,Nizhny Novgorod,0.0,0.0,2006,Baltika
165,1042129,1699795800,1088,Dinamo Moscow,2.0,0.0,1080,Orenburg
166,1042131,1699804800,2012,PFC Sochi,0.0,2.0,1083,Rubin


In [11]:
all_data = data_PL23.merge(data_RFPL23, how='outer').sort_values('timestamp').reset_index(drop=True)
all_data

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,1042016,1689955200,1088,Dinamo Moscow,1.0,3.0,621,Krasnodar
1,1042021,1690020000,1084,Ural,2.0,1.0,555,CSKA Moscow
2,1042020,1690028100,2011,Nizhny Novgorod,0.0,2.0,596,Zenit Saint Petersburg
3,1042017,1690036200,597,Lokomotiv Moscow,2.0,2.0,1083,Rubin
4,1042018,1690045200,1085,Akhmat Grozny,1.0,2.0,1079,Krylya Sovetov
...,...,...,...,...,...,...,...,...
294,1035149,1699797600,66,Aston Villa,3.0,1.0,36,Fulham
295,1035155,1699797600,48,West Ham,3.0,2.0,65,Nottingham Forest
296,1042131,1699804800,2012,PFC Sochi,0.0,2.0,1083,Rubin
297,1035151,1699806600,49,Chelsea,4.0,4.0,50,Manchester City


In [12]:
# 22 год
data_PL22 = pd.read_csv('elo/csv/England/2022/PL.csv')
data_kubokPL22 = pd.read_csv('elo/csv/England/2022/cup.csv')
data_kubokPL22
data_kubokPL22 = data_kubokPL22[data_kubokPL22['home_team_id'].isin(data_PL22['home_team_id'].unique()) & data_kubokPL22['away_team_id'].isin(
                 data_PL22['home_team_id'].unique())]
data_carabaoPL22 = pd.read_csv('elo/csv/England/2022/carabao.csv')
data_carabaoPL22
data_carabaoPL22 = data_carabaoPL22[data_carabaoPL22['home_team_id'].isin(data_PL22['home_team_id'].unique()) & data_carabaoPL22['away_team_id'].isin(
                 data_PL22['home_team_id'].unique())]
data_PL22 = data_PL22.merge(data_kubokPL22, how='outer')
data_PL22 = data_PL22.merge(data_carabaoPL22, how='outer')
data_PL22.sort_values('timestamp', inplace=True)
data_PL22.reset_index(inplace=True, drop=True)
data_PL22

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,867946,1659726000,52,Crystal Palace,0.0,2.0,42,Arsenal
1,867947,1659785400,36,Fulham,2.0,2.0,40,Liverpool
2,867948,1659794400,35,Bournemouth,2.0,0.0,66,Aston Villa
3,867949,1659794400,63,Leeds,2.0,1.0,39,Wolves
4,867951,1659794400,34,Newcastle,2.0,0.0,65,Nottingham Forest
...,...,...,...,...,...,...,...,...
405,868323,1685287800,46,Leicester,2.0,1.0,48,West Ham
406,868324,1685287800,33,Manchester United,2.0,1.0,36,Fulham
407,868316,1685287800,42,Arsenal,5.0,0.0,39,Wolves
408,868318,1685287800,55,Brentford,1.0,0.0,50,Manchester City


In [15]:
data_RFPL22 = pd.read_csv('elo/csv/Russia/2022/RFPL.csv')
data_kubokRFPL22 = pd.read_csv('elo/csv/Russia/2022/KUBOK.csv')
data_kubokRFPL22 = data_kubokRFPL22[data_kubokRFPL22['home_team_id'].isin(data_RFPL22['home_team_id'].unique()) & data_kubokRFPL22['away_team_id'].isin(
                 data_RFPL22['home_team_id'].unique())]
data_RFPL22 = data_RFPL22.merge(data_kubokRFPL22, how='outer').sort_values('timestamp').reset_index(drop=True)
data_RFPL22.reset_index(inplace=True, drop=True)
data_RFPL22

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,879867,1657904400,1994,Khimki,1,1,596,Zenit Saint Petersburg
1,879868,1657972800,555,CSKA Moscow,2,0,1084,Ural
2,879869,1657981800,1080,Orenburg,2,4,1079,Krylya Sovetov
3,879870,1657990800,1085,Akhmat Grozny,1,1,558,Spartak Moscow
4,879871,1658059200,3985,Torpedo Moskva,1,3,2012,PFC Sochi
...,...,...,...,...,...,...,...,...
304,880102,1685800800,597,Lokomotiv Moscow,3,1,3985,Torpedo Moskva
305,880103,1685800800,1085,Akhmat Grozny,2,2,621,Krasnodar
306,880104,1685800800,1080,Orenburg,3,0,1088,Dinamo Moscow
307,880106,1685800800,1084,Ural,1,0,2012,PFC Sochi


In [17]:
#формирование выборки для обучения
general_eng = data_PL22.iloc[100:410].reset_index(drop=True)
general_eng

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,868064,1666204200,40,Liverpool,1.0,0.0,48,West Ham
1,868063,1666204200,34,Newcastle,1.0,0.0,45,Everton
2,868056,1666204200,35,Bournemouth,0.0,1.0,41,Southampton
3,868065,1666206900,33,Manchester United,2.0,0.0,47,Tottenham
4,868060,1666290600,36,Fulham,3.0,0.0,66,Aston Villa
...,...,...,...,...,...,...,...,...
305,868323,1685287800,46,Leicester,2.0,1.0,48,West Ham
306,868324,1685287800,33,Manchester United,2.0,1.0,36,Fulham
307,868316,1685287800,42,Arsenal,5.0,0.0,39,Wolves
308,868318,1685287800,55,Brentford,1.0,0.0,50,Manchester City


In [18]:
general_rus = (data_RFPL22.iloc[80:]).reset_index(drop=True)
general_rus

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,945678,1663070400,2011,Nizhny Novgorod,1,1,1994,Khimki
1,945681,1663079400,1088,Dinamo Moscow,2,1,1085,Akhmat Grozny
2,945677,1663156800,597,Lokomotiv Moscow,2,2,621,Krasnodar
3,945683,1663156800,2012,PFC Sochi,1,3,3985,Torpedo Moskva
4,945684,1663165800,555,CSKA Moscow,2,1,1084,Ural
...,...,...,...,...,...,...,...,...
224,880102,1685800800,597,Lokomotiv Moscow,3,1,3985,Torpedo Moskva
225,880103,1685800800,1085,Akhmat Grozny,2,2,621,Krasnodar
226,880104,1685800800,1080,Orenburg,3,0,1088,Dinamo Moscow
227,880106,1685800800,1084,Ural,1,0,2012,PFC Sochi


In [19]:
all_general = general_eng.merge(general_rus, how='outer')
all_general = all_general.merge(all_data, how='outer')
all_general.sort_values('timestamp').reset_index(drop=True, inplace=True)
all_general

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,868064,1666204200,40,Liverpool,1.0,0.0,48,West Ham
1,868063,1666204200,34,Newcastle,1.0,0.0,45,Everton
2,868056,1666204200,35,Bournemouth,0.0,1.0,41,Southampton
3,868065,1666206900,33,Manchester United,2.0,0.0,47,Tottenham
4,868060,1666290600,36,Fulham,3.0,0.0,66,Aston Villa
...,...,...,...,...,...,...,...,...
833,1035149,1699797600,66,Aston Villa,3.0,1.0,36,Fulham
834,1035155,1699797600,48,West Ham,3.0,2.0,65,Nottingham Forest
835,1042131,1699804800,2012,PFC Sochi,0.0,2.0,1083,Rubin
836,1035151,1699806600,49,Chelsea,4.0,4.0,50,Manchester City


In [20]:
match_data = pd.read_csv('match_data.csv',index_col='id')

In [23]:
#формирование x и y
def create_samples(data):
    y = []
    x = []
    for _, row in data.iterrows():
        new_line_home = []
        new_line_away = []
        home_team_data = match_data[(match_data['team_id'] == row['home_team_id']) & (match_data['time'] < row['timestamp'])][-10::]
        away_team_data = match_data[(match_data['team_id'] == row['away_team_id']) & (match_data['time'] < row['timestamp'])][-10::]
        hh = home_team_data[home_team_data['position'] == 'home']
        ha = home_team_data[home_team_data['position'] == 'away']

        ah = away_team_data[away_team_data['position'] == 'home']
        aa = away_team_data[away_team_data['position'] == 'away']

        new_line_home.extend([row['home_team_id'], row['timestamp'], hh['goals'].mean(), ha['goals'].mean(), hh['shots'].mean(), ha['shots'].mean(), hh['xg'].mean(), ha['xg'].mean()])
        new_line_away.extend([row['away_team_id'], row['timestamp'], ah['goals'].mean(), aa['goals'].mean(), ah['shots'].mean(), aa['shots'].mean(), ah['xg'].mean(), aa['xg'].mean()])

        x.extend([new_line_home, new_line_away]) 
        y.extend([[0.5, min(row['home_goals'], 4) if row['home_goals'] > 0 else 0], [1, min(row['away_goals'], 4) if row['away_goals'] > 0 else 0]])
        #y.extend([[1, row['home_goals']], [2, row['away_goals']]])
    return x, y

In [24]:
x, y = create_samples(all_general)
x = pd.DataFrame(x, columns=['team_id','time', 'mean_goals_home', 'mean_goals_away', 'shots_home', 'shots_away', 'xg_home', 'xg_away'])
y = pd.DataFrame(y, columns=['position', 'actual_goal'])

In [25]:
all_data = pd.concat([x, y], axis=1)
all_data_home = all_data[all_data['position'] == 0.5].reset_index(drop=True)
all_data_away = all_data[all_data['position'] == 1].reset_index(drop=True)
all_data_away

,team_id,time,mean_goals_home,mean_goals_away,shots_home,shots_away,xg_home,xg_away,position,actual_goal
0,48,1666204200,1.2,0.600000,4.20,3.600000,1.2000,0.600000,1.0,0.0
1,45,1666204200,0.6,1.000000,3.80,3.600000,0.6000,1.000000,1.0,0.0
2,41,1666204200,1.2,0.600000,6.00,1.400000,1.2000,0.600000,1.0,1.0
3,47,1666206900,3.0,1.400000,8.00,4.200000,3.0000,1.400000,1.0,0.0
4,66,1666290600,0.8,0.600000,3.60,3.800000,0.8000,0.600000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
833,36,1699797600,1.0,1.000000,3.20,3.400000,1.1440,1.180000,1.0,1.0
834,65,1699797600,1.0,0.833333,2.25,3.500000,0.6400,0.951667,1.0,2.0
835,1083,1699804800,0.4,0.800000,2.60,4.000000,0.4000,0.962000,1.0,2.0
836,50,1699806600,2.5,1.500000,4.50,7.166667,1.6675,1.776667,1.0,4.0


In [27]:
elo_england_22 = pd.read_csv('./elo/csv/England/2022/ratingElo2022_2023PL.csv')
elo_england_23 = pd.read_csv('./elo/csv/England/2023/ratingElo2023_2024PL.csv')

elo_england_22.set_index('id', inplace=True)
elo_england_23.set_index('id', inplace=True)

elo_england_23['prev_elos'] = elo_england_23.apply(lambda row: elo_england_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_england_22.index else row['prev_elos'], axis=1)
elo_england_23.reset_index(inplace=True)

elo_russian_22 = pd.read_csv('./elo/csv/Russia/2022/ratingElo2022_2023RFPL.csv')
elo_russian_23 = pd.read_csv('./elo/csv/Russia/2023/ratingElo2023_2024RFPL.csv')

elo_russian_22.set_index('id', inplace=True)
elo_russian_23.set_index('id', inplace=True)

elo_russian_23['prev_elos'] = elo_russian_23.apply(lambda row: elo_russian_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_russian_22.index else row['prev_elos'], axis=1)
elo_russian_23.reset_index(inplace=True)
all_elo = pd.concat([elo_england_23, elo_russian_23])
all_elo.reset_index(inplace=True)

In [28]:
import re, json
def process_row(row):
    fixed_string = re.sub(r'\}\{', ',', all_elo[all_elo['id'] == row['team_id']]['prev_elos'].iloc[0])
    fixed_string = re.sub(r'(\d+)(?=:)', r'"\1"', fixed_string)
    json_string = json.loads(fixed_string)
    return json_string[f"{int(row['time'])}"]

In [29]:
def add_column_elo(frame):
    frame['elo'] = frame.apply(process_row, axis=1)

In [30]:
samples = [all_data_home, all_data_away]
for item in samples:
    add_column_elo(item)

In [31]:
all_data_home['diff_elo'] = all_data_home['elo'] - all_data_away['elo']
all_data_away['diff_elo'] = all_data_away['elo'] - all_data_home['elo']

In [35]:
all_data_home.corr()

,team_id,time,mean_goals_home,mean_goals_away,shots_home,shots_away,xg_home,xg_away,position,actual_goal,elo,diff_elo
team_id,1.000000,-0.014123,-0.214822,-0.194943,-0.239398,-0.227625,-0.248244,-0.206901,NaN,-0.134765,-0.768986,-0.239557
time,-0.014123,1.000000,-0.075618,0.030922,-0.156269,-0.062619,-0.066574,0.080439,NaN,0.004286,0.031675,-0.002928
mean_goals_home,-0.214822,-0.075618,1.000000,0.361706,0.673239,0.376275,0.899682,0.318227,NaN,0.175609,0.445108,0.479367
mean_goals_away,-0.194943,0.030922,0.361706,1.000000,0.300293,0.631611,0.356512,0.902442,NaN,0.209363,0.352914,0.418569
shots_home,-0.239398,-0.156269,0.673239,0.300293,1.000000,0.376596,0.733647,0.273642,NaN,0.179139,0.397456,0.377219
shots_away,-0.227625,-0.062619,0.376275,0.631611,0.376596,1.000000,0.399934,0.632473,NaN,0.207668,0.355970,0.390399
xg_home,-0.248244,-0.066574,0.899682,0.356512,0.733647,0.399934,1.000000,0.340839,NaN,0.161910,0.457462,0.459170
xg_away,-0.206901,0.080439,0.318227,0.902442,0.273642,0.632473,0.340839,1.000000,NaN,0.205072,0.362526,0.410039
position,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
actual_goal,-0.134765,0.004286,0.175609,0.209363,0.179139,0.207668,0.161910,0.205072,NaN,1.000000,0.191002,0.314400


In [93]:
# создаем отложенную выборку
x_train_home, x_test_home = train_test_split(
    all_data_home.drop(['team_id', 'time','position' ,'actual_goal', 'elo'], axis=1), train_size=0.75, shuffle=True, random_state=1,
)
y_train_home, y_test_home = train_test_split(
    all_data_home["actual_goal"], train_size=0.75, shuffle=True, random_state=1,
)

# фирксируем стратегию кросс-валидации
kfold = KFold(n_splits=10
                   , shuffle=True, random_state=25)

In [92]:
x_train_away, x_test_away = train_test_split(
    all_data_away.drop(['team_id', 'time','position' ,'actual_goal', 'elo'], axis=1), train_size=0.75, shuffle=True, random_state=1,
)
y_train_away, y_test_away = train_test_split(
    all_data_away["actual_goal"], train_size=0.75, shuffle=True, random_state=1,
)

In [40]:
def get_graphics_goals(y_test, y_pred, msg):
    plt.plot(y_test, label='Actual')
    plt.plot(y_pred, label='Predicted')
    plt.xlabel('Sample')
    plt.ylabel('Value')
    plt.title(msg)
    plt.legend()
    plt.show()

In [108]:
model_linear = Ridge(alpha=1.0, max_iter=100000)
model_forest = RandomForestRegressor(random_state=25, max_depth=2, min_samples_leaf=12, n_estimators=300, min_samples_split=3)
modelPoly = make_pipeline(MinMaxScaler(), PolynomialFeatures(degree=2, include_bias=False), Ridge(random_state=25, alpha=1.0))

In [118]:
# проводим кросс-валидацию
cv_home = -1 * cross_val_score(
    estimator=model_forest,
    X=x_train_home,
    y=y_train_home,
    scoring="neg_mean_absolute_error",
    cv=kfold
)
print(f"CV-results: {round(np.mean(cv_home), 4)} +/- {round(np.std(cv_home), 3)}")


CV-results: 0.9288 +/- 0.095


In [119]:
# проводим кросс-валидацию
cv_away = -1 * cross_val_score(
    estimator=model_forest,
    X=x_train_away,
    y=y_train_away,
    scoring="neg_mean_absolute_error",
    cv=kfold
)
print(f"CV-results: {round(np.mean(cv_away), 4)} +/- {round(np.std(cv_away), 3)}")

CV-results: 0.8699 +/- 0.059


In [120]:
(np.mean(cv_home) + (np.mean(cv_away))) / 2

0.89934630448569

In [ ]:
def print_neuron_results(model, x, y, label, label_gra):
    test_loss, test_mae = model.evaluate(x, y)
    y_pred = model.predict(x)
    print(label)
    print(f"Test Loss (MSE): {test_loss}")
    #print(f"Test RMSE: {np.sqrt(test_loss_home)}")
    print(f"Test MAE: {test_mae}")
    print(f"MIN: {min(y_pred)}")
    print(f"MAX: {max(y_pred)}")
    get_graphics_goals(y, y_pred, label_gra)

In [131]:
def cross_validate_neural_network(x, y, model, n_splits=5, epochs=1000, batch_size=32, patience=70):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=25)
    mae_scores = []

    for train_index, val_index in kfold.split(x, y):
        x_train, x_val = x.iloc[train_index], x.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

        history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=True,
                            validation_data=(x_val, y_val), callbacks=[early_stopping], verbose=0)

        val_predictions = model.predict(x_val)
        mae = mean_absolute_error(y_val, val_predictions)
        mae_scores.append(mae)

    return mae_scores


In [126]:
def create_neural_network(input_shape):
    model = keras.Sequential([
        keras.layers.Dense(343, activation='linear', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(343, activation='linear', kernel_regularizer=l2(0.01)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(49, activation='linear', kernel_regularizer=l2(0.01)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(7, activation='relu', kernel_regularizer=l2(0.01)),
        keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    return model

In [127]:
input_shape = (7, )
model_home = create_neural_network(input_shape)
model_away = create_neural_network(input_shape)

In [132]:
mae_scores_home = cross_validate_neural_network(x_train_home, y_train_home, model_home)
mae_scores_away = cross_validate_neural_network(x_train_away, y_train_away, model_away)

# Print the mean and standard deviation of MAE scores for each network
print("Mean MAE for Home Network:", np.mean(mae_scores_home))
print("Std MAE for Home Network:", np.std(mae_scores_home))

print("Mean MAE for Away Network:", np.mean(mae_scores_away))
print("Std MAE for Away Network:", np.std(mae_scores_away))


4/4 [==============================] - 0s 4ms/step
Mean MAE for Home Network: 0.9787728214591269
Std MAE for Home Network: 0.09627277119183225
Mean MAE for Away Network: 0.8801859220822653
Std MAE for Away Network: 0.04385849525148081
